In [ ]:
from pymongo import ASCENDING, DESCENDING

def helper_filter(collection, orderBy=None, limitToFirst=None, limitToLast=None, equalTo=None, startAt=None, endAt=None):
    
    filter = {}
    sort_order = None

    if orderBy:
        if orderBy == '$key':
            key = '_id'
        elif orderBy == '$value':
            raise ValueError("orderBy '$value' not supported. Please use a specific field name.")
        else:
            key = orderBy
            # Create index for the orderBy field
            collection.create_index([(key, ASCENDING)])

        if startAt is not None:
            filter[f'{key}'] = {'$gte': startAt}

        if endAt is not None:
            if f'{key}' in filter:
                filter[f'{key}']['$lte'] = endAt
            else:
                filter[f'{key}'] = {'$lte': endAt}

        if equalTo is not None:
            filter[f'{key}'] = equalTo

        if limitToFirst is not None:
            limit = limitToFirst
            sort_order = ASCENDING
        elif limitToLast is not None:
            limit = limitToLast
            sort_order = DESCENDING
        else:
            limit = None

    return filter, limit, sort_order

def get(collection, documentFilter, jsonPath, filter=None):
    if filter is None:
        if jsonPath == '':
            result = list(collection.find(documentFilter))
        else: 
            result = collection.find(documentFilter).distinct(jsonPath)
    else:
        filter, limit, sort_order = helper_filter(collection, **filter)
        if jsonPath == '':
            if sort_order is not None:
                result = list(collection.find({'$and': [documentFilter, filter]}).sort('_id', sort_order).limit(limit))
            else:
                result = list(collection.find({'$and': [documentFilter, filter]}))
        else:
            if sort_order is not None:
                result = collection.find({'$and': [documentFilter, filter]}).sort('_id', sort_order).distinct(jsonPath)[:limit]
            else:
                result = collection.find({'$and': [documentFilter, filter]}).distinct(jsonPath)
    return result